# First Pytorch Code: Practial Neural Network with Pytorch

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
data = pd.read_csv('./data/diabetes.csv')

data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [3]:
x = data.iloc[:, 0:-1].values
y = list(data.iloc[:, -1].values)

In [4]:
x.shape

(768, 7)

In [5]:
y[0:10]

['positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive']

In [6]:
y_int = [1 if string == 'positive' else 0 for string in y]

In [7]:
len(y_int)

768

In [8]:
y = np.array(y_int, dtype='float64')
y

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0.

In [9]:
y.shape

(768,)

In [10]:
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ..., -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -0.69289057,
        -1.10325546, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.73518964, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.69289057,
        -0.24020459,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.69289057,
        -0.20212881, -0.87137393]])

In [11]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [12]:
x.shape, y.shape

(torch.Size([768, 7]), torch.Size([768, 1]))

In [13]:
# custom dataset class
class Dataset(Dataset):
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [14]:
dataset = Dataset(x, y)


In [15]:
len(dataset)

768

In [16]:
from torch.utils.data import DataLoader

In [17]:
BATCH_SIZE = 32
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

In [18]:
train_loader

In [19]:
print(f"There is {len(train_loader)} batches in the dataset")
for (x, y) in train_loader:
    print(f"For one iteration (batch) there is :")
    print(f"Data {x.shape}")
    print(f"Labels {y.shape}")
    break

There is 24 batches in the dataset
For one iteration (batch) there is :
Data torch.Size([32, 7])
Labels torch.Size([32, 1])


In [20]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out


In [21]:
net = Model(7, 1)

In [22]:
loss = torch.nn.BCELoss()
optim = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

In [23]:
# training
EPOCHS = 200

for epoch in range(EPOCHS):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()

        outputs = net(inputs)
        train_loss = loss(outputs, labels)
        optim.zero_grad() # clear gradient to prevent gradient accomulation
        train_loss.backward() #backward pass
        optim.step() #grdient step

    # accuracy calculation
    out = (outputs > 0.5).float()
    acc = (out == labels).float().mean()
    if epoch % 10 == 0:
        print(f"EPOCH {epoch+1}/{EPOCHS}, Loss: {train_loss:.3f}, Accuracy: {acc:.3f}")


EPOCH 1/200, Loss: 0.606, Accuracy: 0.625
EPOCH 11/200, Loss: 0.444, Accuracy: 0.812
EPOCH 21/200, Loss: 0.513, Accuracy: 0.781
EPOCH 31/200, Loss: 0.462, Accuracy: 0.719
EPOCH 41/200, Loss: 0.498, Accuracy: 0.750
EPOCH 51/200, Loss: 0.525, Accuracy: 0.719
EPOCH 61/200, Loss: 0.497, Accuracy: 0.781
EPOCH 71/200, Loss: 0.504, Accuracy: 0.656
EPOCH 81/200, Loss: 0.219, Accuracy: 0.938
EPOCH 91/200, Loss: 0.423, Accuracy: 0.719
EPOCH 101/200, Loss: 0.395, Accuracy: 0.781
EPOCH 111/200, Loss: 0.188, Accuracy: 0.938
EPOCH 121/200, Loss: 0.435, Accuracy: 0.719
EPOCH 131/200, Loss: 0.344, Accuracy: 0.844
EPOCH 141/200, Loss: 0.476, Accuracy: 0.781
EPOCH 151/200, Loss: 0.493, Accuracy: 0.750
EPOCH 161/200, Loss: 0.525, Accuracy: 0.656
EPOCH 171/200, Loss: 0.356, Accuracy: 0.750
EPOCH 181/200, Loss: 0.701, Accuracy: 0.562
EPOCH 191/200, Loss: 0.568, Accuracy: 0.719


### NOTE:
Here the purpose of this code is not to find the optimal values for model parameter but to accustomed overselves with pytorch code. This is the first part of our Pytorch learning and long journey is waiting ahead of us.